In [49]:
import json

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

In [50]:
octo = pd.read_csv('bad-scrape.raw.csv', names=[i for i in range(0, 4)])

In [51]:
octo_test = octo.copy()

In [52]:
# blank col, name col, value col, blank col
rows = octo_test.loc[octo_test[0].isnull() & octo_test[3].isnull()].index.values
octo_test.iloc[rows, 0] = octo_test.iloc[rows][1]
octo_test.iloc[rows, 1] = octo_test.iloc[rows][2]

In [53]:
# blank col, name col, blank col, value col
rows = octo_test.loc[octo_test[0].isnull() & octo_test[2].isnull()].index.values
octo_test.iloc[rows, 0] = octo_test.iloc[rows][1]
octo_test.iloc[rows, 1] = octo_test.iloc[rows][3]

In [54]:
# blank col, blank col, name col, value col
rows = octo_test.loc[octo_test[0].isnull() & octo_test[1].isnull()].index.values
octo_test.iloc[rows, 0] = octo_test.iloc[rows][2]
octo_test.iloc[rows, 1] = octo_test.iloc[rows][3]

In [55]:
# name col, blank col, blank col, value col
octo_test.loc[octo_test[1].isnull() & octo_test[2].isnull(), 1] = octo_test.loc[octo_test[1].isnull() & octo_test[2].isnull()][3]

In [56]:
# name col, blank col, value col, blank col
octo_test.loc[octo_test[0].notnull() & octo_test[1].isnull() & octo_test[2].notnull(), 1] = octo_test.loc[octo_test[0].notnull() & octo_test[1].isnull() & octo_test[2].notnull()][2]

In [57]:
# remove indoor/outdoor rows
octo_test[0] = octo_test[0].str.rstrip()
octo_test.drop(list(octo_test.loc[octo_test[0].isin(['Indoor', 'Outdoor', 'Oudoor'])].index.values), inplace=True)

# remove dummy cols
octo_test.drop([2, 3], axis=1, inplace=True)

# reset the index
octo_test.reset_index(inplace=True, drop=True)

In [58]:
def process(df):
    # cleanup
    df.rename(columns={0: 'fixture_location', 1: 'result_value'}, inplace=True)
    df['park_name'] = ['' for i in range(0, len(df))]
    
    # get park names
    indices = df.loc[df.result_value.isnull()].index
    names = df.loc[df.result_value.isnull()].fixture_location
    parks = list(zip(indices, names))

    # restructure
    for ind, park in enumerate(parks):
        start, park = park
        try:
            end = parks[ind+1][0] - 1
        except IndexError:
            end = len(df)
        df.loc[start:end, 'park_name'] = park
        
    df.drop(list(indices), inplace=True)
    df['result_flag'] = df['result_value'].apply(lambda x: x.split(' ')[-1] if len(x.split(' ')) > 1 else '')
    df['result_value'] = df['result_value'].apply(lambda x: x.split(' ')[0]).str.replace('˂', '<')

In [59]:
process(octo_test)
octo_test = octo_test.ix[:, [2, 0, 1, 3]]

In [60]:
# octo_test.loc[octo_test['fixture_location'].str.contains('\*'), 'offline'] = 'Y'
# octo_test.loc[octo_test['result_value'].str.contains('<'), 'threshold'] = 'Y'

In [61]:
# clean up two rows w/o space between result value and result flag
octo_test.iloc[1002]['result_value'] = '3.09'
octo_test.iloc[1002]['result_flag'] = 'J'
octo_test.iloc[1002]['processed_result'] = 3.09

octo_test.iloc[2020]['result_value'] = '0.243'
octo_test.iloc[2020]['result_flag'] = 'J'
octo_test.iloc[2020]['processed_result'] = 0.243

In [62]:
octo_test['processed_result'] = octo_test['result_value'].apply(lambda x: 0 if '<' in x else pd.to_numeric(x))

In [63]:
# epa threshold is worded 'if lead _exceeds_ 15 ppb'
octo_test['exceeds_epa'] = octo_test['processed_result'].apply(lambda x: 1 if x > 15 else 0)

In [64]:
octo_test['result_flag'] = octo_test['result_flag'].str.strip().replace('', np.nan)

In [65]:
octo_test.loc[octo_test.park_name.str.contains('Logan'), 'park_name'] = 'Logan Square Skate Park'

In [70]:
for string in ('Follow Up', 'Follow-Up', 'repair', 'Test 2'):
    octo_test.drop(octo_test.loc[octo_test.fixture_location.str.contains(string)].index.values, inplace=True)

In [86]:
pd.DataFrame(
    {'score': octo_test.groupby(['park_name'])['exceeds_epa'].mean(),
     'num_fixtures': octo_test.groupby(['park_name'])['fixture_location'].count()}
).reset_index()

,park_name,num_fixtures,score
0,Abbott,7,0.000000
1,Ada,6,0.000000
2,Adams (George & Adelle),5,0.400000
3,Adams (John),1,0.000000
4,Addam/Medil,1,0.000000
5,Addams Memorial (Jane),1,0.000000
6,Aiello,3,0.000000
7,Algonquin,3,0.000000
8,Altgeld,6,0.000000
9,Amundsen,4,0.500000


In [83]:
octo_test.to_csv('chi-parks-lead-results-10-2016.processed.csv', index=False)